<a href="https://colab.research.google.com/github/hajihye123/MachineLearning_edwith/blob/main/7_%EC%95%99%EC%83%81%EB%B8%94_%ED%95%99%EC%8A%B5_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from sklearn.datasets import load_iris, load_wine, load_boston, load_diabetes
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate, cross_val_score

* Cross-Validation
  * 과적합을  피하기 위해서 사용
  * 훈련 데이터 전체를 한 번에 사용하지 않고, 일부를 남겨두고 테스트에 활용하는 것, 교차 검증
  * 큰 값이 좋다
* cross_val_score
  * 'cross_val_score(학습에 사용될 모델, X, Y, cv=반복횟수)'
  * cross_val_score 에선 큰 값이 좋은 결과를 나타냄
  * 회귀 모델에선 MSE를 얻기 위해 주로 'neg_mean_squared_error' 값을 사용(0으로 갈수록 좋음)

In [28]:
iris = load_iris()
wine = load_wine()
boston = load_boston()
diabetes = load_diabetes()

# 투표기반 분류와 회귀

In [29]:
from sklearn.ensemble import VotingClassifier, VotingRegressor

## 투표 기반 분류

In [30]:
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier

In [31]:
model1 = SVC()
model2 = GaussianNB()
model3 = SGDClassifier()

In [32]:
vote_model = VotingClassifier(
    estimators=[('svc', model1), ('guassian', model2), ('sgd', model3)]
)

In [33]:
for model in (model1, model2, model3, vote_model):
  model_name = str(type(model)).split('.')[-1][:-2]
  scores = cross_val_score(model, iris.data, iris.target, cv=5)
  print(f'{model_name}, Accuracy: {scores.mean()}')

SVC, Accuracy: 0.9666666666666666
GaussianNB, Accuracy: 0.9533333333333334
SGDClassifier, Accuracy: 0.78
VotingClassifier, Accuracy: 0.9666666666666668


## 투표 기반 회귀

In [34]:
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor, LinearRegression

In [35]:
model1 = SVR()
model2 = SGDRegressor()
model3 = LinearRegression()

In [36]:
vote_model = VotingRegressor(
    estimators = [('svr', model1), ('sgd', model2), ('linear', model3)]
)

In [37]:
for model in (model1, model2, model3, vote_model):
  model_name = str(type(model)).split('.')[-1][:-2]
  scores = cross_val_score(model, boston.data, boston.target, scoring='neg_mean_squared_error', cv=5)
  print(f'{model_name}, Accuracy: {scores.mean()}') # 왜 음수..? # 0에 가까우면 좋음

SVR, Accuracy: -71.85800739156483
SGDRegressor, Accuracy: -6.1833888869471455e+28
LinearRegression, Accuracy: -37.13180746769903
VotingRegressor, Accuracy: -9.194327856039402e+26


# 배깅

In [38]:
from sklearn.ensemble import BaggingClassifier, BaggingRegressor

## 배깅을 사용한 분류

In [44]:
from sklearn.preprocessing import StandardScaler

base_model = make_pipeline(
    StandardScaler(),
    SVC()
)

bagging_model = BaggingClassifier(base_model, n_estimators=10, max_samples=0.5, max_features=0.5)

base_cross_val = cross_validate(estimator=base_model, X=wine.data, y=wine.target, cv=5)
print(f"{base_cross_val['test_score'].mean()}")

bagging_cross_val = cross_validate(estimator=bagging_model, X=wine.data, y=wine.target, cv=5)
print(f"{bagging_cross_val['test_score'].mean()}")

0.9833333333333334
0.9665079365079364


## 배깅을 사용한 회귀

In [45]:
from sklearn.preprocessing import StandardScaler

base_model = make_pipeline(
    StandardScaler(),
    SVR()
)

bagging_model = BaggingRegressor(base_model, n_estimators=10, max_samples=0.5, max_features=0.5)

base_cross_val = cross_validate(estimator=base_model, X=boston.data, y=boston.target, cv=5)
print(f"{base_cross_val['test_score'].mean()}")

bagging_cross_val = cross_validate(estimator=bagging_model, X=boston.data, y=boston.target, cv=5)
print(f"{bagging_cross_val['test_score'].mean()}")

0.17631266230186618
0.16482351909806708


# AdaBoost

In [46]:
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor

## AdaBoost 분류

In [47]:
model = make_pipeline(
    StandardScaler(),
    AdaBoostClassifier()
)

In [48]:
cross_val = cross_validate(estimator=model, X=iris.data, y=iris.target, cv=5)
print(f"{cross_val['test_score'].mean()}")

0.9466666666666667


In [49]:
cross_val = cross_validate(estimator=model, X=wine.data, y=wine.target, cv=5)
print(f"{cross_val['test_score'].mean()}")

0.8085714285714285


## Adaboost 회귀

In [51]:
model = make_pipeline(
    StandardScaler(),
    AdaBoostRegressor()
)

In [52]:
cross_val = cross_validate(estimator=model, X=boston.data, y=boston.target, cv=5)
print(f"{cross_val['test_score'].mean()}")

0.5932095461428804
